In [ ]:
import os
import numpy as np
import pandas as pd

from sklearn import model_selection

import tensorflow as tf
import tflearn

from sklearn.metrics import r2_score

# Reading the Data

In [ ]:
datapath = 'Data/'
test_csv = 'test.csv'
train_csv = 'train.csv'
sample_csv = 'sample_submission.csv'

In [ ]:
data = pd.read_csv(os.path.join(datapath,train_csv))

# Understanding the Data

In [ ]:
print('Variables with letters are categorical. Variables with 0/1 are binary values.')
data.head(3)

In [ ]:
tp_int = []
tp_float = []
tp_obj = []
tp_other = []

#Getting all variables names, less ID and looking for it types 
for i in data.columns.to_series()[1:]:
    if data[i].dtype == 'int64':
        tp_int.append(i)
    elif data[i].dtype == 'float64':
        tp_float.append(i)
    elif data[i].dtype == 'object':
        tp_obj.append(i)
    else:
        tp_other.append(i)
        
dic = {'tp_int': tp_int, 'tp_float': tp_float, 'tp_obj': tp_obj, 'tp_other': tp_other}

print('Categorical:', tp_obj)
print('Float:', tp_float)
print('Has other type?', tp_other)

In [ ]:
drop = []
binarys = []
for i in tp_int:
    print('Variable: {}, Min: {}, Max: {} , Unique: {}'.format(i, data[i].min(), data[i].max() ,data[i].unique()))
    if data[i].max() == 0:
        drop.append(i)
    elif data[i].max() == 1 and data[i].min() == 0 and len(data[i].unique()) == 2:
        binarys.append(i)
    else:
        pass 

In [ ]:
print('Variables only with 0:', drop)

if len(drop) + len(binarys) == len(tp_int):
    print('All other integer variables are binary')

In [ ]:
#Dropping variables only with zero
data = data.drop(drop, axis = 1)

# One hot enconding

In [ ]:
#Best way to work with categorical variables? I don't think is the best in all case. I will study about that

for each in tp_obj:
    dummies = pd.get_dummies(data[each], prefix=each, drop_first=False)
    data = pd.concat([data, dummies], axis=1)

data = data.drop(tp_obj, axis=1)
data.head()

# Test Data

In [ ]:
data_test = pd.read_csv(os.path.join(datapath,test_csv))

In [ ]:
#Dropping variables only with zero
data_test = data_test.drop(drop, axis = 1)

In [ ]:
for each in tp_obj:
    dummies = pd.get_dummies(data_test[each], prefix=each, drop_first=False)
    data_test = pd.concat([data_test, dummies], axis=1)

data_test = data_test.drop(tp_obj, axis=1)
data_test.head()

# Keeping only columns who are in both data sets

In [ ]:
keep = np.intersect1d(data.columns.to_series(), data_test.columns.to_series())

In [ ]:
data = data[np.append(keep,'y')]

In [ ]:
data_test = data_test[keep]

# Splitting data into training/testing

In [ ]:
train_features, test_features, train_y, test_y = model_selection.train_test_split(
    data, data['y'], test_size = 0.3, random_state = 0)

In [ ]:
train_features = train_features.drop(['ID', 'y'], axis = 1).values
test_features = test_features.drop(['ID', 'y'], axis = 1).values

train_y = train_y[:, np.newaxis]

In [ ]:
test_features.shape

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

randomforest = RandomForestRegressor(n_estimators=200, max_features='auto', bootstrap=False, 
                                   oob_score=False, n_jobs=-1, random_state=0).fit(train_features, train_y)

randomforest_score = randomforest.score(test_features, test_y)
print('RF Score:', randomforest_score)

predict = randomforest.predict(test_features)

slope, intercept, r_value, p_value, std_err = stats.linregress(predict, test_y)
print('R Square:', r_value**2)

In [ ]:
predict_test = randomforest.predict(data_test.iloc[:, 1:])

In [ ]:
submission = pd.DataFrame({'ID': data_test['ID'], 'y': predict_test})
submission.to_csv(os.path.join(datapath,'submissionrf.csv'), index = False)

# Rede Neural

In [ ]:
import tensorflow as tf
import tflearn

# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, train_features.shape[1]])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 800, activation='ReLU')
    net = tflearn.dropout(net, 0.80)  
    
    net = tflearn.fully_connected(net, 1200, activation='ReLU')
    net = tflearn.dropout(net, 0.80)
    
    net = tflearn.fully_connected(net, 800, activation='ReLU')
    net = tflearn.dropout(net, 0.80)

    net = tflearn.fully_connected(net, 400, activation='ReLU')
    net = tflearn.dropout(net, 0.80)    
    
    # Output layer and training model
    net = tflearn.fully_connected(net, 1, activation='linear')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss="mean_square")
    
    model = tflearn.DNN(net)
    return model

model = build_model()

In [ ]:
import tensorflow as tf
import tflearn

# Define the neural network
def build_model():
    # This resets all parameters and variables, leave this here
    tf.reset_default_graph()
    
    # Inputs
    net = tflearn.input_data([None, train_features.shape[1]])

    # Hidden layer(s)
    net = tflearn.fully_connected(net, 512, activation='ReLU') 
    net = tflearn.fully_connected(net, 512, activation='ReLU')
    net = tflearn.dropout(net, 0.80)
    
    # Output layer and training model
    net = tflearn.fully_connected(net, 1, activation='linear')
    net = tflearn.regression(net, optimizer='sgd', learning_rate=0.1, loss="mean_square")
    
    model = tflearn.DNN(net)
    return model

model = build_model()

In [ ]:
model.fit(train_features, train_y, validation_set=0.2, show_metric=True, batch_size=512, n_epoch=10000)

In [ ]:
predict = model.predict(test_features)
predict = [predict[i][0] for i in range(0, len(predict))]

slope, intercept, r_value, p_value, std_err = stats.linregress(predict, test_y)
print('R Square:', r_value**2)

In [ ]:
predict_test = model.predict(data_test.iloc[:, 1:])
predict_test = [predict_test[i][0] for i in range(0, len(predict_test))]

In [ ]:
submission = pd.DataFrame({'ID': data_test['ID'], 'y': predict_test})
print(submission.head())
submission.to_csv(os.path.join(datapath,'submissiondp.csv'), index = False)

# xgboost

In [ ]:
 ()# mmm, xgboost, loved by everyone ^-^
import xgboost as xgb

# prepare dict of params for xgboost to run with
xgb_params = {
    'n_trees': 500, 
    'eta': 0.005,
    'max_depth': 4,
    'subsample': 0.95,
    'objective': 'reg:linear',
    'eval_metric': 'rmse',
    'base_score': train_y.mean(), # base prediction = mean(target)
    'silent': 1
}

# form DMatrices for Xgboost training
dtrain = xgb.DMatrix(train_features, train_y)
dtest = xgb.DMatrix(data_test.iloc[:, 1:].values)

# xgboost, cross-validation
cv_result = xgb.cv(xgb_params, 
                   dtrain, 
                   num_boost_round=500, # increase to have better results (~700)
                   early_stopping_rounds=50,
                   verbose_eval=50, 
                   show_stdv=False
                  )

num_boost_rounds = len(cv_result)
print(num_boost_rounds)

# train model
model = xgb.train(dict(xgb_params, silent=0), dtrain, num_boost_round=num_boost_rounds)

In [ ]:
print(r2_score(dtrain.get_label(), model.predict(dtrain)))

In [ ]:
# make predictions and save results
y_pred = model.predict(dtest)
submission = pd.DataFrame({'id': data_test['ID'], 'y': y_pred})
submission.to_csv(os.path.join(datapath,'submissionxg.csv'), index = False)